In [1]:
using ITensors
using PyCall

np = pyimport("numpy")

using Plots
pyplot()

Plots.PyPlotBackend()

In [47]:
h = 1.
J = 1

Lx = 3
Ly = 3

L  = Lx*Ly

sites = siteinds("S=1/2",L)
psi0 = randomMPS(sites)
ampo = OpSum()

for i=1:L
    val = mod(i,Ly)
    if mod(i,Ly) != 0
        #println("$i $val")
        if i != L
            ampo += -4*J,"Sz",i,"Sz",i+1
        end
    end
    if i <= (Lx-1)*Ly
        val = i + Ly
        println("$i $val")
        ampo += -4*J,"Sz",i,"Sz",i+Ly
    end
    ampo += -2*h,"Sx",i
end

H = MPO(ampo,sites)

sweeps = Sweeps(10)
setmaxdim!(sweeps, 10,20,100,200,300,300,400,500,600,1000)
setcutoff!(sweeps, 1E-17)

@show sweeps

#z_observer = DMRGObserver(["Sz","Sx"],sites,energy_tol=1E-12)

energy, psi = dmrg(H, psi0, sweeps)#,observer=Sz_observer)
println("Final energy = $energy")

1 4
2 5
3 6
4 7
5 8
6 9
sweeps = Sweeps
1 cutoff=1.0E-17, maxdim=10, mindim=1, noise=0.0E+00
2 cutoff=1.0E-17, maxdim=20, mindim=1, noise=0.0E+00
3 cutoff=1.0E-17, maxdim=100, mindim=1, noise=0.0E+00
4 cutoff=1.0E-17, maxdim=200, mindim=1, noise=0.0E+00
5 cutoff=1.0E-17, maxdim=300, mindim=1, noise=0.0E+00
6 cutoff=1.0E-17, maxdim=300, mindim=1, noise=0.0E+00
7 cutoff=1.0E-17, maxdim=400, mindim=1, noise=0.0E+00
8 cutoff=1.0E-17, maxdim=500, mindim=1, noise=0.0E+00
9 cutoff=1.0E-17, maxdim=600, mindim=1, noise=0.0E+00
10 cutoff=1.0E-17, maxdim=1000, mindim=1, noise=0.0E+00

After sweep 1 energy=-13.764411239928 maxlinkdim=4 maxerr=0.00E+00 time=0.006
After sweep 2 energy=-13.813534889499 maxlinkdim=16 maxerr=0.00E+00 time=0.007
After sweep 3 energy=-13.814889727919 maxlinkdim=16 maxerr=0.00E+00 time=0.023
After sweep 4 energy=-13.816868706038 maxlinkdim=16 maxerr=0.00E+00 time=0.009
After sweep 5 energy=-13.818383858397 maxlinkdim=16 maxerr=0.00E+00 time=0.007
After sweep 6 energy=-13.

In [41]:
np.array(H.data[26].tensor)

12×12×2×2 Array{Float64,4}:
[:, :, 1, 1] =
  1.0       0.0   0.0        0.0  0.0  …  0.0  0.0  0.0   0.0        0.0
  0.0       1.0  -2.75276    0.0  0.0     0.0  0.0  0.0   0.0       -0.649839
  0.425325  0.0   0.0        0.0  0.0     0.0  0.0  0.0  -0.525731   0.0
  0.0       0.0   0.850651   0.0  0.0     0.0  0.0  0.0   0.0       -0.525731
  0.0       0.0   0.0       -1.0  0.0     0.0  0.0  0.0   0.0        0.0
  0.0       0.0   0.0        0.0  1.0  …  0.0  0.0  0.0   0.0        0.0
  0.0       0.0   0.0        0.0  0.0     0.0  0.0  0.0   0.0        0.0
  0.0       0.0   0.0        0.0  0.0     0.0  0.0  0.0   0.0        0.0
  0.0       0.0   0.0        0.0  0.0     1.0  0.0  0.0   0.0        0.0
  0.0       0.0   0.0        0.0  0.0     0.0  1.0  0.0   0.0        0.0
  0.0       0.0   0.0        0.0  0.0  …  0.0  0.0  1.0   0.0        0.0
 -0.262866  0.0   0.0        0.0  0.0     0.0  0.0  0.0  -0.850651   0.0

[:, :, 2, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 

In [51]:
using TimeEvoMPS

┌ Info: Precompiling TimeEvoMPS [1bec0e0a-d6ff-11e9-2fd9-7928ca571592]
└ @ Base loading.jl:1278


In [55]:
N = L
psi = productMPS(sites, fill("↑",N))
cb = LocalMeasurementCallback(["Sz","Sx","Sy"], sites, 0.2)
tdvp!(psi,H,0.1,5,maxdim=30,callback=cb)

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter C:\Users\asant\.julia\packages\ProgressMeter\Vf8un\src\ProgressMeter.jl:620
Evolving state... 100%|█████████████████████████████████| Time: 0:01:49
  t:             5.0
  dt step time:  1.854
  Max bond-dim:  30


In [ ]:
dag(psi)*H*psi